# APS 2

## Imports

In [2]:
from math import cosh, sinh, sqrt, tanh, log, pi
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

## Declarações

In [5]:

rho_mat = 2700                            # Densidade do material em kg/m³
c_mat = 896                               # Calor especícico do material em J/(kg.K)
k_mat = 180                               # Condutividade térmica do material em W/(m.K)
h_amb = 50                                # Coeficiente de transf. por convecção do ambiente em W/(m².K)
t_fluido = 50                             # Temperatura do fluido em ºC
t_base = 100                              # Temperatura da base do dissipador em ºC
t_extremidade = 25                        # Temperatura da extremidade do dissipador em ºC
raio =  5e-3/2                            # Raio do dissipador
comprimento = 0.3                         # Comprimento do dissipador em m
p = 2*pi*raio                             # Perímetro do dissipador
a = pi*raio**2                            # Área da secção transversal do dissipador
alpha = k_mat/(rho_mat*c_mat)             # Difusividade térmica do material em m²/s



dx =  0.03                                # delta x
dt =  0.01                                # delta t

max_dt = ((dx**2)/(alpha*(((h_amb*p*dx**2)/(k_mat*a))+2)))*(0.9) # Cálculo do delta t máximo
tol = 1e-10                               # Tolerância para o erro
t_total = 10                              # Tempo total de simulação
max_dt

4.948363636363635

$\frac{d^2T}{dx^2} - \frac{hP}{K \cdot A_{tr}} \cdot (T-T_{\infty}) = \frac{1}{\alpha}\cdot \frac{dT}{dt}$

Substituindo a derivada segunda $\frac{d^2T}{dx^2}$ pela equação de diferenças finitas centrada e a derivada primeira $\frac{dT}{dt}$ pela equação de diferença progressiva, temos:

$\frac{T_{m+1} - 2T_m + T_{m-1}}{\Delta x^2} - \frac{hP}{K \cdot A_{tr}} \cdot (T-T_{\infty}) = \frac{1}{\alpha}\cdot \frac{T_m ^{\ l+1} - T_m ^{\ l}}{\Delta t}$

Deixando em função de $T_m ^{\ l+1}$, temos:

$T_m ^{\ l+1} = \left( \frac{\alpha \cdot \Delta t}{\Delta x^{\ 2}}\right) \cdot \left(  T_{m+1} -2T_m + T_{m-1}    \right) - \left(  \frac{h \cdot P \cdot \alpha \cdot \Delta t}{k \cdot A_{tr}}  \right) \cdot \left( T - T_{\infty} \right)  +  T_m ^l$


In [6]:
def sol_analitica(temperaturas):
    """
    Permite prever a solução analítica do problema.
    """
    novas_temperaturas = []
    for t in temperaturas:
        t_analitica = 
    

SyntaxError: invalid syntax (1803088307.py, line 7)